In [1]:
import pandas

In [12]:
#coords_csv = pandas.read_csv('tioga-emerson.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/china/Henan/Henan QGIS files/henan-vertices-updated.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/china/huizhou-lng-terminal/pipeline-route-trace.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/usa/north-bakken/north-bakken-route-trace.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/usa/improving-usa-routes/georeference-images/carolina-gas-transmission-route.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/usa/improving-usa-routes/eastern-gas-transmission-collected.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/usa/improving-usa-routes/georeference-images/port-arthur-pipeline.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/europe/georeferenced-images/goleniow-route.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/usa/improving-usa-routes/florida-gas-transmission-co.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/australia/csv-files/wgp.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/australia/csv-files/central-west-pipeline.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/australia/csv-files/central-ranges-pipeline.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/usa/improving-usa-routes/csv-files/anr-pipeline-simplified.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/usa/improving-usa-routes/csv-files/enable-gas-transmission-simplified-007.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/usa/improving-usa-routes/csv-files/atmos-texas-simlified002.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/australia/csv-files/vic-trans-system-001.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/russia/power-of-siberia-1.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/canada/ngtl-vertices-simplified.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/australia/australia-route-fixing-gympie.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/israel/israel-egypt-pipelines.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/australia/australia-route-fixing-bassgas.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/australia/australia-route-fixing.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/gfit-pipelines/mapping/australia/australia-route-fixing-darling-downs.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/report-gas-pipelines-2022/figures-and-mapping/from-scott/mountaineer-pipeline.csv')
#coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/pipelines/mapping/2022-oil/tazama/tazama-pipeline.csv')
coords_csv = pandas.read_csv('/Users/baird/Dropbox/_gem-docs/projects/pipelines/mapping/2022-oil/report-mapping/willow-pipeline.csv')

* look at the csv file to understand how it's organized
* ultimately each x,y pair should look like:  x1,y1 : x2,y2 : x3,y3 : ...

In [13]:
coords_csv

,id,name,vertex_index,vertex_part,vertex_part_index,distance,angle,lon,lat
0,NaN,lokchar-lamu,0,0,0,0.000000,144.999363,35.636902,2.364136
1,NaN,lokchar-lamu,1,0,1,0.014376,137.144385,35.645148,2.352360
2,NaN,lokchar-lamu,2,0,2,0.032504,126.992280,35.659178,2.340881
3,NaN,lokchar-lamu,3,0,3,0.072837,123.618453,35.692339,2.317923
4,NaN,lokchar-lamu,4,0,4,0.141143,122.771256,35.749921,2.281180
...,...,...,...,...,...,...,...,...,...
1282,NaN,willow7,36,0,36,0.196222,275.144687,-151.963645,70.133342
1283,NaN,willow7,37,0,37,0.203013,284.376170,-151.970367,70.134313
1284,NaN,willow7,38,0,38,0.206313,296.028794,-151.973457,70.135470
1285,NaN,willow7,39,0,39,0.208209,305.134772,-151.975073,70.136462


* create a list of pipeline names that match up exactly with the csv file; this is something you'll edit yourself

* _aside: to pull out all the rows where the Name column matches "Example Pipeline 1", use the syntax below:_

* now loop through each ```pipeline_name``` in this list and do the appropriate reformatting, and store that in ```list_of_formatted_coords```

### if from the EIA dataset, the pipelines don't have individual names

in QGIS, select the given pipeline(s), then Vector > Collect geometries, then add $x $y (lon, lat) and export as a csv file; then you'll use the "vertex_part" label as the pipeline name

### if normal separate segments

In [14]:
segment_name = 'name'
#segment_name = 'PipelineName'
#segment_name = 'LINE_NO'
#segment_name = 'OBJECTID'

In [15]:
#list_of_pipelines = ["Tioga to Emerson Pipeline"]
list_of_pipelines = list(sorted(set(coords_csv[segment_name])))
#list_of_pipelines = list(set(coords_csv['PipelineName']))
print(list_of_pipelines)
print(sorted(list(list_of_pipelines)))

['basra-aqaba', 'kenya-to-lokichar-segment', 'lokchar-lamu', 'new-mundra-panipat', 'paradip-numaligarh', 'vankor1', 'vankor2', 'vankor3', 'willow1', 'willow2', 'willow3', 'willow4', 'willow5', 'willow6', 'willow7']
['basra-aqaba', 'kenya-to-lokichar-segment', 'lokchar-lamu', 'new-mundra-panipat', 'paradip-numaligarh', 'vankor1', 'vankor2', 'vankor3', 'willow1', 'willow2', 'willow3', 'willow4', 'willow5', 'willow6', 'willow7']


In [16]:
list_of_formatted_coords = []
for pipeline_name in list_of_pipelines:
    # pull out all rows matching pipeline_name
#    pipeline_specific_dataframe = coords_csv.loc[coords_csv['PipelineName']==pipeline_name]
    pipeline_specific_dataframe = coords_csv.loc[coords_csv[segment_name]==pipeline_name]
    # then change y,x coords into strings
    pipeline_specific_dataframe[['lat','lon']].astype(str)
    # join those strings with a comma
    joined_strings_dataframe = pipeline_specific_dataframe[['lat', 'lon']].astype(str).apply(lambda x: ','.join(x), axis=1)
    # add the colon
    joined_strings_with_colon = joined_strings_dataframe.str.cat(sep=':')
    # store this in a list
    list_of_formatted_coords.append(joined_strings_with_colon)

In [17]:
' ; '.join(list_of_formatted_coords).__len__()

26361

In [21]:
for index,pipeline_name in enumerate(list_of_pipelines):
    if 'willow' in pipeline_name:
        print(pipeline_name, '--', list_of_formatted_coords[index])
        print()

    #print(pipeline_name, '--', list_of_formatted_coords[index])
    #print()

willow1 -- 70.28941,-149.881002:70.287876,-149.88723:70.28138,-149.895525:70.278695,-149.925768:70.2771,-149.948669:70.275792,-149.965565:70.274895,-149.977665:70.274705,-149.989405:70.274081,-149.994529:70.273001,-149.998896:70.270943,-150.00552:70.269983,-150.008611:70.268894,-150.01256:70.268141,-150.01854:70.267956,-150.030693:70.267969,-150.045319:70.267801,-150.058304:70.267618,-150.066276:70.26745,-150.072784:70.266706,-150.080685:70.264771,-150.096992:70.262365,-150.112511:70.258998,-150.132936:70.255699,-150.156798:70.251909,-150.179007:70.245897,-150.218831:70.244702,-150.227224:70.244167,-150.230763:70.243817,-150.235997:70.243794,-150.242473:70.243942,-150.257551:70.244197,-150.270454:70.244298,-150.275616:70.244215,-150.279081:70.243537,-150.283072:70.242246,-150.28673:70.240671,-150.291295:70.240134,-150.294832:70.239472,-150.299682:70.239297,-150.313945:70.238902,-150.332557:70.238538,-150.352887:70.238433,-150.37102:70.238279,-150.378808:70.237236,-150.387596:70.235973,

In [19]:
for index,pipeline_name in enumerate(list_of_pipelines):
    print(';'); print(list_of_formatted_coords[index])

;
29.529646,35.005821:29.506322,35.108447:29.502833,35.166045:29.500917,35.208193:29.504749,35.229268:29.512412,35.24651:29.53157,35.275248:29.567971,35.311648:29.629278,35.367208:29.742312,35.478326:29.799787,35.533885:29.836188,35.562623:29.880252,35.587528:29.918569,35.599023:29.964549,35.606687:29.997118,35.616266:30.046929,35.633508:30.079499,35.646919:30.117815,35.664162:30.177206,35.696731:30.240429,35.727384:30.305567,35.769533:30.366874,35.807849:30.453086,35.853829:30.535467,35.892146:30.648501,35.947705:30.763451,35.987938:30.820926,36.007096:30.878401,36.018591:30.972277,36.037749:31.087227,36.05116:31.250073,36.056908:31.322875,36.056908:31.391845,36.05116:31.466562,36.043497:31.533616,36.026254:31.568101,36.016675:31.600671,36.012844:31.644735,36.016675:31.691826,36.029468:31.755232,36.054126:31.832729,36.092874:31.917271,36.159803:31.987722,36.223209:32.027812,36.257814:32.065312,36.30201:32.086741,36.331475:32.097456,36.36094:32.100134,36.390404:32.097456,36.41853:32.08

### if it's a big network AND segments are not separated, you gotta deal with the "vertex_part" info

In [21]:
list_of_pipelines = ["CNGC's Inter-Provincial Gas Pipeline Network"]

In [32]:
list_of_formatted_coords = []

for pipeline_name in list_of_pipelines: 
    list_of_vertex_parts = list(set(coords_csv[coords_csv['PipelineName']==pipeline_name]['vertex_part']))
    
    for vertex_part in list_of_vertex_parts:
        # pull out all rows matching pipeline_name
        pipeline_specific_dataframe = coords_csv.loc[(coords_csv['PipelineName']==pipeline_name)&(coords_csv['vertex_part']==vertex_part)]
        # then change y,x coords into strings
        pipeline_specific_dataframe[['lat','lon']].astype(str)
        # join those strings with a comma
        joined_strings_dataframe = pipeline_specific_dataframe[['lat', 'lon']].astype(str).apply(lambda x: ', '.join(x), axis=1)
        # add the colon
        joined_strings_with_colon = joined_strings_dataframe.str.cat(sep=':')
        # store this in a list
        list_of_formatted_coords.append(joined_strings_with_colon)

* finally, write some code to spit out the information in the format you want

In [12]:
len(list_of_formatted_coords)

242

In [34]:
for i in list_of_formatted_coords:
    print(i)
    print(';')

11.038248, 103.796843:11.080021, 103.877933:11.168483, 103.99834:11.202885, 104.076972:11.217629, 104.12366:11.286432, 104.221951:11.340492, 104.315328:11.360151, 104.344815:11.387181, 104.386589:11.419125, 104.418533:11.438783, 104.462764:11.473185, 104.541397:11.480557, 104.615115:11.502672, 104.654432:11.566562, 104.676547:11.659938, 104.683919:11.755772, 104.737979:11.819661, 104.750265:11.854063, 104.715863:11.898294, 104.708492:11.972012, 104.720778:12.035901, 104.72815:12.121905, 104.696205:12.171051, 104.654432:12.205453, 104.651974:12.242312, 104.669175:12.259513, 104.661803:12.276714, 104.632316:12.274256, 104.590542:12.311116, 104.56597:12.333231, 104.53894:12.362718, 104.504538:12.379919, 104.477508:12.42415, 104.45785:12.465924, 104.443106:12.475753, 104.354644:12.507697, 104.285841:12.53227, 104.212122:12.53227, 104.0696:12.53227, 103.931993:12.542099, 103.890219:12.5593, 103.858275:12.571587, 103.818959:12.652677, 103.708381:12.674792, 103.629748:12.709194, 103.548658:12